# <center> 流畅的Python </center>
## 第十四章：可迭代的对象、迭代器和生成器
**示例14-1 把句子划分为单词序列**

In [1]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    

**示例14-2 测试Sentence实例能否迭代**

In [5]:
s = Sentence('The time has come, the Walrus said, ')
print(s)
for word in s:
    print(word)
print(list(s))
print(s[0])

Sentence('The time has...Walrus said, ')
The
time
has
come
the
Walrus
said
['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']
The


**可迭代对象和迭代器的对比**

In [6]:
s = 'ABC'
for ch in s:
    print(ch)
print('\n')

it = iter(s)
while(True):
    try:
        print(next(it))
    except StopIteration:
        del(it)
        break

A
B
C


A
B
C


**示例14-3 abc.Iterator类**

In [9]:
from collections.abc import Iterable

class Iterator(Iterable):

    __slots__ = ()

    @abstractmethod
    def __next__(self):
        'return the next item from the iterator. When exhausted, raise StopIteration'
        raise StopIteration
        
    def __iter__(self):
        return self
    
    @classmethod
    def __subclasshook__(cls, C):
        if(cls is Iterator):
            if(any('__next__' in B.__dict__ for B in C.__mro__) and 
               any('__iter__' in B.__dict__ for B in C.__mro__)):
                return True
        return NotImplemented
    

NameError: name 'abstractmethod' is not defined

**构建迭代器**

In [11]:
s3 = Sentence('Pig and Pepper')
it = iter(s3)
print(it)
print(next(it))
print(next(it))
print(next(it))
#print(next(it))
print(list(it))
print(list(iter(s3)))

Pig
and
Pepper
[]
['Pig', 'and', 'Pepper']


**示例14.4 使用迭代器模式实现Sentence类**

In [14]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return SentenceIterator(self.words)
    
class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

**测试**

In [15]:
s = Sentence('The time has come, the Walrus said, ')
print(s)
for word in s:
    print(word)
print(list(s))
print(s[0])

Sentence('The time has...Walrus said, ')
The
time
has
come
the
Walrus
said
['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']
The


**示例14-5 使用生成器函数实现Sentence类**

In [16]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words:
            yield word

**测试**

In [17]:
s = Sentence('The time has come, the Walrus said, ')
print(s)
for word in s:
    print(word)
print(list(s))
print(s[0])

Sentence('The time has...Walrus said, ')
The
time
has
come
the
Walrus
said
['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']
The


**说明生成器的行为**

In [18]:
def gen123():
    yield 1
    yield 2
    yield 3
    
print(gen123)
print(gen123())
for i in gen123():
    print(i)
print('\n')

g = gen123()
print(next(g))
print(next(g))
print(next(g))
print(next(g))

<function gen123 at 0x10f4f2f28>
<generator object gen123 at 0x10f4dd7d8>
1
2
3


1
2
3


StopIteration: 

**示例14-6 运行时打印消息的生成器函数**

In [19]:
def genAB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end')
    
for c in genAB():
    print('--> ', c, '\n')

start
-->  A 

continue
-->  B 

end


**示例14-7 在生成器函数中调用re.finditer生成器函数**

In [20]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in RE_WORD.finditer(self.text):
            yield word.group()

**测试**

In [21]:
s = Sentence('The time has come, the Walrus said, ')
print(s)
for word in s:
    print(word)
print(list(s))

Sentence('The time has...Walrus said, ')
The
time
has
come
the
Walrus
said
['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']


**示例14-8 生成器表达式**

In [24]:
def genAB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end')
    
res1 = [x*3 for x in genAB()]
print('\n')
for i in res1:
    print('-->', i)

print('\n')
res2 = (x*3 for x in genAB())
for i in res2:
    print('-->', i, '\n')

start
continue
end


--> AAA
--> BBB


start
--> AAA 

continue
--> BBB 

end


**示例14-9 使用生成器表达式实现**

In [25]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

**测试**

In [26]:
s = Sentence('The time has come, the Walrus said, ')
print(s)
for word in s:
    print(word)
print(list(s))


Sentence('The time has...Walrus said, ')
The
time
has
come
the
Walrus
said
['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']


**示例14-11 ArithmeticProgression类**

In [7]:
class ArithmeticProgression:
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end
        
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)
        forever = self.end is None
        index = 0
        while(forever or result < self.end):
            yield result
            index += 1
            result = self.begin + self.step * index
            

**示例14-10 演示ArithmeticProgression**

In [8]:
ap = ArithmeticProgression(0, 1, 3)
print(list(ap))
ap = ArithmeticProgression(1, .5, 3)
print(list(ap))
ap = ArithmeticProgression(0, 0.33, 1)
print(list(ap))

from fractions import Fraction
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
print(list(ap))
from decimal import Decimal
ap = ArithmeticProgression(0, Decimal('.1'), .3)
print(list(ap))

[0, 1, 2]
[1.0, 1.5, 2.0, 2.5]
[0.0, 0.33, 0.66, 0.99]
[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]
[Decimal('0'), Decimal('0.1'), Decimal('0.2')]


**示例14-12 aritprog_gen生成器函数**

In [9]:
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while(forever or result < end):
        yield result
        index += 1
        result = begin + step * index

**测试**

In [12]:
ap = aritprog_gen(0, 1, 3)
print(list(ap))
ap = aritprog_gen(1, .5, 3)
print(list(ap))
ap = aritprog_gen(0, 0.33, 1)
print(list(ap))

[0, 1, 2]
[1.0, 1.5, 2.0, 2.5]
[0.0, 0.33, 0.66, 0.99]


**使用itertools模块生成等差数列**

In [13]:
import itertools

gen = itertools.count(1, .5)
for i in range(5):
    print(next(gen))

1
1.5
2.0
2.5
3.0


**示例14-13 与前面的airtprog函数作用相同**

In [16]:
import itertools

def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if(end is not None):
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

**测试**

In [17]:
ap = aritprog_gen(0, 1, 3)
print(list(ap))
ap = aritprog_gen(1, .5, 3)
print(list(ap))
ap = aritprog_gen(0, 0.33, 1)
print(list(ap))

[0, 1, 2]
[1.0, 1.5, 2.0, 2.5]
[0.0, 0.33, 0.66, 0.99]


**示例14-14 演示用于过滤的生成器函数**

In [31]:
from itertools import *

def vowel(c):
    return c.lower() in 'aeiou'

print('original:    ', list('Aardvark'), '\n')
print('filter:      ', list(filter(vowel, 'Aardvark')))
print('filterfalse: ', list(filterfalse(vowel, 'Aardvark')))
print('dropwhile:   ', list(dropwhile(vowel, 'Aardvark')))
print('compress:    ', list(compress('Aardvark', (1, 0, 1, 1, 0, 1))))
print('islice:      ', list(islice('Aardvark', 4)))
print('islice:      ', list(islice('Aardvark', 4, 7)))
print('islice:      ', list(islice('Aardvark', 1, 7, 2)))

original:     ['A', 'a', 'r', 'd', 'v', 'a', 'r', 'k'] 

filter:       ['A', 'a', 'a']
filterfalse:  ['r', 'd', 'v', 'r', 'k']
dropwhile:    ['r', 'd', 'v', 'a', 'r', 'k']
compress:     ['A', 'r', 'd', 'a']
islice:       ['A', 'a', 'r', 'd']
islice:       ['v', 'a', 'r']
islice:       ['a', 'd', 'a']


**示例14-15 演示itertools.accumulate生成器函数**

In [53]:
from itertools import accumulate

def print_list(sample):
    for item in sample:
        print('{:8d}'.format(item), end='')
    print('\n')

sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
print('ori: ', end='')
print_list(sample)

print('sum: ', end='')
print_list(list(accumulate(sample)))

print('min: ', end='')
print_list(list(accumulate(sample, min)))
print('max: ', end='')
print_list(list(accumulate(sample, max)))

import operator
print('mul: ', end='')
print_list(accumulate(sample, operator.mul))

print('mul: ', end='')
print_list(accumulate(range(1, 11), operator.mul))

ori:        5       4       2       8       7       6       3       0       9       1

sum:        5       9      11      19      26      32      35      35      44      45

min:        5       4       2       2       2       2       2       0       0       0

max:        5       5       5       8       8       8       8       8       9       9

mul:        5      20      40     320    2240   13440   40320       0       0       0

mul:        1       2       6      24     120     720    5040   40320  362880 3628800



**示例14-16 演示用于映射的生成器函数**

In [60]:
from itertools import *

print(list(enumerate('albatroz', 1)))

import operator
print(list(map(operator.mul, range(11), range(11))))
print(list(map(lambda a, b: (a, b), range(11), [2, 4, 8])))
print(list(starmap(operator.mul, enumerate('albatroz', 1))))

sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
print(list(starmap(lambda a, b: b/a, enumerate(accumulate(sample), 1))))

[(1, 'a'), (2, 'l'), (3, 'b'), (4, 'a'), (5, 't'), (6, 'r'), (7, 'o'), (8, 'z')]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
[(0, 2), (1, 4), (2, 8)]
['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']
[5.0, 4.5, 3.6666666666666665, 4.75, 5.2, 5.333333333333333, 5.0, 4.375, 4.888888888888889, 4.5]


**示例14-17 演示用于合并的生成器函数**

In [65]:
from itertools import *

print(list(chain('ABC', range(2))))
print(list(chain(enumerate('ABC'))))
print(list(zip('ABC', range(5), [10, 20, 30, 40])))
print(list(zip_longest('ABC', range(5))))
print(list(zip_longest('ABC', range(5), fillvalue='?')))

['A', 'B', 'C', 0, 1]
[(0, 'A'), (1, 'B'), (2, 'C')]
[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]
[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]
[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]


**示例14-18 演示itertools.product生成器函数**

In [74]:
from itertools import product

print(list(product('ABC', range(2))))
suits = 'Spades hearts diamonds clubs'.split()
print(list(product('AK', suits)))
print(list(product('ABC')))
print(list(product('ABC', repeat=2)))
print(list(product(range(2), repeat=3)))

rows = product('AB', range(2), repeat=2)
for row in rows:
    print(row)

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]
[('A', 'Spades'), ('A', 'hearts'), ('A', 'diamonds'), ('A', 'clubs'), ('K', 'Spades'), ('K', 'hearts'), ('K', 'diamonds'), ('K', 'clubs')]
[('A',), ('B',), ('C',)]
[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'B'), ('B', 'C'), ('C', 'A'), ('C', 'B'), ('C', 'C')]
[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


**示例14-19 演示count, repeat和cycle的用法**

In [81]:
import itertools

ct = itertools.count()
print(next(ct))
print(next(ct), next(ct), next(ct))

print(list(itertools.islice(itertools.count(1, .3), 3)))

cy = itertools.cycle('ABC')
print(next(cy))
print(list(itertools.islice(cy, 7)))

rp = itertools.repeat(7)
print(next(rp), next(rp))
print(list(itertools.repeat(8, 4)))
print(list(map(operator.mul, range(11), itertools.repeat(5))))

0
1 2 3
[1, 1.3, 1.6]
A
['B', 'C', 'A', 'B', 'C', 'A', 'B']
7 7
[8, 8, 8, 8]
[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]


**示例14-20 组合学生成器函数会从输入的各个元素中产出多个值**

In [85]:
print(list(itertools.combinations('ABC', 2)))
print(list(itertools.combinations_with_replacement('ABC', 2)))
print(list(itertools.permutations('ABC', 2)))
print(list(itertools.product('ABC', repeat=2)))

[('A', 'B'), ('A', 'C'), ('B', 'C')]
[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]
[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]
[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'B'), ('B', 'C'), ('C', 'A'), ('C', 'B'), ('C', 'C')]


**示例14-21 groupby用法**

In [90]:
print(list(itertools.groupby('LLLLAAGGG')))

for char, group in itertools.groupby('LLLLAAGGG'):
    print(char, '->', list(group))
print('\n')

animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear', 'bat', 'dolphin', 'shark', 'lion']
animals.sort(key=len)
for length, group in itertools.groupby(animals, len):
    print(length, '->', list(group))

print('\n')
for length, group in itertools.groupby(reversed(animals), len):
    print(length, '->', list(group))

[('L', <itertools._grouper object at 0x10f4699b0>), ('A', <itertools._grouper object at 0x10f469080>), ('G', <itertools._grouper object at 0x10f4698d0>)]
L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A']
G -> ['G', 'G', 'G']


3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['giraffe', 'dolphin']


7 -> ['dolphin', 'giraffe']
5 -> ['shark', 'eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']


**示例14-22 tee函数**

In [92]:
print(list(itertools.tee('ABC')))
g1, g2 = itertools.tee('ABC')
print(next(g1), next(g2), next(g2))
print(list(g1))
print(list(g2))

print(list(zip(*itertools.tee('ABC'))))

[<itertools._tee object at 0x10f6217c8>, <itertools._tee object at 0x10f6813c8>]
A A B
['B', 'C']
['C']
[('A', 'A'), ('B', 'B'), ('C', 'C')]


**示例14-23 all and any**

In [93]:
print(all([1, 2, 0]))
print(any([1, 2, 0]))

False
True
